# Traffic information Netherlands
The intention of this Python script is to scrape actual travel information about expected delays.
The first part will do this for traffic, the second part can do this for trains.

Source: https://www.anwb.nl/feeds/gethf

### Init all necessary libraries

In [ ]:
# Load packages into memory
import requests
import json
import pandas as pd
import seaborn as sns
from datetime import datetime

from pandas.io.json import json_normalize

from tinydb import TinyDB

dbname='traffic_db.json'

### Run scraper of data independently
```python collect.py```

### Retrieve collected raw data

In [ ]:
# Let's get the current VAT rate in the EU
dbname='traffic_db.json'
db = TinyDB(dbname)

all_data = db.all()

# Pandas version 0.25 contains new parameter max_level in json_normalize.
pd.__version__

### Translate JSON data into dataframe (separate parsing from data collection)

In [ ]:
traffic_data = pd.DataFrame()

for jso in all_data:# Create json object from returned text
    #jso = json.loads(routeInfo.text)
    # Create dataframe from json object roadEntries
    trafficdf = json_normalize(jso['roadEntries'])

    # Drop unnecessary columns (I did not find any documentation about the webservice yet to only retrieve what I need.)
    trafficdf.drop(columns = ['events.roadWorks','events.radars'],inplace=True)

    # Read the number of trafficJams and filter the dataframe on it.
    trafficdf['listlength'] = trafficdf['events.trafficJams'].apply(lambda x : len(x))
    trafficdf = trafficdf[trafficdf['listlength'] > 0]
    trafficdf.drop(columns = ['listlength'], inplace=True)
    trafficdf['timestap'] = datetime.now()

    # Pop all trafficjams, normalize and join back to dataframe to keep road info.
    trafficdf = (pd.concat({i: json_normalize(x) for i, x in trafficdf.pop('events.trafficJams').items()}, sort=True)
             .reset_index(level=1, drop=True)
             .join(trafficdf)
             .reset_index(drop=True))

    # Drop all items where no delays are mentioned.
    trafficdf = trafficdf.dropna()
    trafficdf.drop(columns=['events'],inplace=True)

    # Set index to msgNr.
    #trafficdf.set_index('msgNr', inplace=True)
    traffic_data=traffic_data.append(trafficdf)
    #trafficdf.tail()

### Have a look at trafficinfo

In [ ]:
print(traffic_data.count())

traffic_data.tail()

In [ ]:
traffic_data.info()

In [ ]:
latestTimestamp = traffic_data.timestap.max()
#dftest[dftest['timestap']==latestTimestamp].plot(x='start',y='delay')
#dftest.plot(x='timestap',y='delay')
traffic_data['delayMinutes'] = traffic_data['delay'] / 60
bp = sns.barplot(x='road',y='delayMinutes',data=traffic_data[traffic_data['timestap']==latestTimestamp])
#bp.set_xticklabels(bp.get_xticklabels(), rotation=90)
bp.set_title('Actual Delays')

### Push dataframe to Azure SQL, based on index

params = 'DRIVER={SQL Server};' \
         'SERVER=****;' \
         'PORT=1433;' \
         'DATABASE=o365data;' \
         'UID=****;' \
         'PWD=****;'
            
params = urllib.parse.quote_plus(params)

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

%time trafficdf.to_sql(name='TrafficInformation',con=engine , schema='dbo', if_exists='append', index=False)